In [ ]:
import ipyparallel as ipp

cluster = ipp.Cluster(engines="mpi", n=6).start_and_connect_sync()
%autopx

In [ ]:
from mpi4py import MPI

mpi_comm = MPI.COMM_WORLD
rank = mpi_comm.Get_rank()

In [ ]:
%%capture
from pace.util import CubedSphereCommunicator, CubedSpherePartitioner, TilePartitioner

layout = (1, 1)
partitioner = CubedSpherePartitioner(TilePartitioner(layout))
communicator = CubedSphereCommunicator(mpi_comm, partitioner)

In [ ]:
from pace.util import QuantityFactory, SubtileGridSizer

backend = "numpy"

sizer = SubtileGridSizer.from_tile_params(
    nx_tile=128,
    ny_tile=128,
    nz=79,
    n_halo=3,
    extra_dim_lengths={},
    layout=layout,
    tile_partitioner=partitioner.tile,
    tile_rank=communicator.tile.rank,
)

quantity_factory = QuantityFactory.from_backend(sizer=sizer, backend=backend)

In [ ]:
from datetime import datetime

restart_path = "/home/ajdas/pace/restart_data/v5.0"
restart_time = datetime.strptime("2020-08-26 12:00:01", "%Y-%m-%d %H:%M:%S")
fortran_data = True

#serial_path = "/home/ajdas/pace/test_data/8.1.1/c12_6ranks_baroclinic_dycore_microphysics/driver/"
#serialized_grid = True

In [ ]:
%%capture
from pace.driver.initialization import RestartConfig

restart_config = RestartConfig(path=restart_path, start_time=restart_time, fortran_data=fortran_data)
fortran_state = restart_config.get_driver_state(quantity_factory, communicator)

In [ ]:
import numpy as np
from pace.util import Quantity

lon_driver = Quantity(
    data = fortran_state.grid_data.lon.data * 180/np.pi,
    dims = ("x_interface", "y_interface"),
    units = "degree",
    origin = (3, 3),
    extent = (129, 129),
    gt4py_backend=backend,
)

lat_driver = Quantity(
    data = fortran_state.grid_data.lat.data * 180/np.pi,
    dims = ("x_interface", "y_interface"),
    units = "degree",
    origin = (3, 3),
    extent = (129, 129),
    gt4py_backend=backend,
)

area_driver = Quantity(
    data = fortran_state.grid_data.area.data,
    dims = ("x", "y"),
    units = "degree",
    origin = (3, 3),
    extent = (128, 128),
    gt4py_backend=backend,
)

lon_driver_global = communicator.gather(lon_driver)
lat_driver_global = communicator.gather(lat_driver)
area_driver_global = communicator.gather(area_driver)

In [ ]:
from netCDF4 import Dataset
import numpy as np

datadir = "/home/ajdas/pace/restart_data/v5.0/"

data = Dataset(datadir + "grid_spec.tile%s.nc" % (rank+1), "r")
lon_edges = np.array(data["grid_lon"])
lat_edges = np.array(data["grid_lat"])
lon_centers = np.array(data["grid_lont"])
lat_centers = np.array(data["grid_latt"])
area = np.array(data["area"])
#data.close()

zero_array = np.zeros(lat_driver.data.shape)

In [ ]:
lon_fortran = Quantity(
    data = zero_array,
    dims = ("x_interface", "y_interface"),
    units = "degree",
    origin = (3, 3),
    extent = (129, 129),
    gt4py_backend=backend,
)
lon_fortran.data[3:-3, 3:-3] = lon_edges

lat_fortran = Quantity(
    data = zero_array,
    dims = ("x_interface", "y_interface"),
    units = "degree",
    origin = (3, 3),
    extent = (129, 129),
    gt4py_backend=backend,
)
lat_fortran.data[3:-3, 3:-3] = lat_edges

lona_fortran = Quantity(
    data = zero_array,
    dims = ("x", "y"),
    units = "degree",
    origin = (3, 3),
    extent = (128, 128),
    gt4py_backend=backend,
)
lona_fortran.data[3:-4, 3:-4] = lon_centers

lata_fortran = Quantity(
    data = zero_array,
    dims = ("x", "y"),
    units = "degree",
    origin = (3, 3),
    extent = (128, 128),
    gt4py_backend=backend,
)
lata_fortran.data[3:-4, 3:-4] = lat_centers

area_fortran = Quantity(
    data = zero_array,
    dims = ("x", "y"),
    units = "degree",
    origin = (3, 3),
    extent = (128, 128),
    gt4py_backend=backend,
)
area_fortran.data[3:-4, 3:-4] = area

lon_fortran_global = communicator.gather(lon_fortran)
lat_fortran_global = communicator.gather(lat_fortran)
lona_fortran_global = communicator.gather(lona_fortran)
lata_fortran_global = communicator.gather(lata_fortran)
area_fortran_global = communicator.gather(area_fortran)

In [ ]:
import matplotlib.pyplot as plt
from cartopy import crs as ccrs
from fv3viz import pcolormesh_cube

if rank == 0:
    # fig = plt.figure(figsize=(12, 6))
    # fig.patch.set_facecolor("white")
    # ax = fig.add_subplot(111, projection=ccrs.Robinson(central_longitude=180))
    # ax.set_facecolor(".4")

    # # plot computational mesh
    # f1 = pcolormesh_cube(
    #     lat_driver_global.data,
    #     lon_driver_global.data,
    #     area_driver_global.data * 2e-11,
    #     cmap="viridis",
    #     vmin=0,
    #     vmax=1,
    # )
    # plt.colorbar(f1, label="m2 x 10^11")

    # ax.set_title("Grid area - driver")


    fig = plt.figure(figsize=(12, 6))
    fig.patch.set_facecolor("white")
    ax = fig.add_subplot(111, projection=ccrs.Robinson(central_longitude=180))
    ax.set_facecolor(".4")
    ax.set_global()


    # plot computational mesh
    # f1 = pcolormesh_cube(
    #     lat_fortran_global.data,
    #     lon_fortran_global.data,
    #     area_fortran_global.data * 2e-11,
    #     cmap="viridis",
    #     vmin=0,
    #     vmax=1,
    # )

    for tile in range(6):   
        ax.plot(lon_fortran_global.view[:][tile][0, :], lat_fortran_global.view[:][tile][0, :], c='k', lw=2, transform=ccrs.PlateCarree(), zorder=10)
        ax.plot(lon_fortran_global.view[:][tile][-1, :], lat_fortran_global.view[:][tile][-1, :], c='k', lw=2, transform=ccrs.PlateCarree(), zorder=10)
        ax.plot(lon_fortran_global.view[:][tile][:, 0], lat_fortran_global.view[:][tile][:, 0], c='k', lw=2, transform=ccrs.PlateCarree())
        ax.plot(lon_fortran_global.view[:][tile][:, -1], lat_fortran_global.view[:][tile][:, -1], c='k', lw=2, transform=ccrs.PlateCarree())


    for tile in range(6):
        ax.text(lon_fortran_global.data[tile][65, 65], lat_fortran_global.data[tile][65, 65], str(tile+1), transform=ccrs.PlateCarree(), c='w')
    #plt.colorbar(f1, label="m2 x 10^11")

    ax.set_title("Grid area - fortran restart data")

    plt.show()

In [ ]:
data = Dataset("/home/ajdas/pace/test_data/8.1.1/c12_6ranks_standard_restart_15min/Grid-Info.nc", "r")

In [ ]:
data

In [ ]:
import pace.util.grid.generation
import importlib

importlib.reload(pace.util.grid.generation)
from pace.util.grid.generation import MetricTerms

metric_terms = MetricTerms.from_lat_lon(lat=lat_edges, lon=lon_edges, quantity_factory=quantity_factory, communicator=communicator)

In [ ]:
import numpy as np
from pace.util import Quantity

lon_mt = Quantity(
    data = metric_terms.lon.data * 180/np.pi,
    dims = ("x_interface", "y_interface"),
    units = "degree",
    origin = (3, 3),
    extent = (129, 129),
    gt4py_backend=backend,
)

lat_mt = Quantity(
    data = metric_terms.lat.data * 180/np.pi,
    dims = ("x_interface", "y_interface"),
    units = "degree",
    origin = (3, 3),
    extent = (129, 129),
    gt4py_backend=backend,
)

lona_mt = Quantity(
    data = metric_terms.lon_agrid.data * 180/np.pi,
    dims = ("x_interface", "y_interface"),
    units = "degree",
    origin = (3, 3),
    extent = (128, 128),
    gt4py_backend=backend,
)

lata_mt = Quantity(
    data = metric_terms.lat_agrid.data * 180/np.pi,
    dims = ("x_interface", "y_interface"),
    units = "degree",
    origin = (3, 3),
    extent = (128, 128),
    gt4py_backend=backend,
)

area_mt = Quantity(
    data = metric_terms.area.data,
    dims = ("x", "y"),
    units = "degree",
    origin = (3, 3),
    extent = (128, 128),
    gt4py_backend=backend,
)

lon_mt_global = communicator.gather(lon_mt)
lat_mt_global = communicator.gather(lat_mt)
lona_mt_global = communicator.gather(lona_mt)
lata_mt_global = communicator.gather(lata_mt)
area_mt_global = communicator.gather(area_mt)

In [ ]:
metric_terms_o = MetricTerms(quantity_factory=quantity_factory, communicator=communicator)
nx1, ny1 = lon_edges.shape
nh = 3

lon_edges_rad = np.deg2rad(lon_edges)
lat_edges_rad = np.deg2rad(lat_edges)
lon_centers_rad = np.deg2rad(lon_centers)
lat_centers_rad = np.deg2rad(lat_centers)

metric_terms_o._grid.data[nh:nh+ny1, nh:nh+nx1, 0] = lon_edges_rad 
metric_terms_o._grid.data[nh:nh+ny1, nh:nh+nx1, 1] = lat_edges_rad 
metric_terms_o._init_agrid()

lon_mto = Quantity(
    data = metric_terms_o.lon.data,
    dims = ("x_interface", "y_interface"),
    units = "degree",
    origin = (nh, nh),
    extent = (nx1, ny1),
    gt4py_backend=backend,
)
lat_mto = Quantity(
    data = metric_terms_o.lat.data,
    dims = ("x_interface", "y_interface"),
    units = "degree",
    origin = (nh, nh),
    extent = (nx1, ny1),
    gt4py_backend=backend,
)

lona_mto = Quantity(
    data = metric_terms_o.lon_agrid.data,
    dims = ("x", "y"),
    units = "degree",
    origin = (nh, nh),
    extent = (nx1-1, ny1-1),
    gt4py_backend=backend,
)
lata_mto = Quantity(
    data = metric_terms_o.lat_agrid.data,
    dims = ("x", "y"),
    units = "degree",
    origin = (nh, nh),
    extent = (nx1-1, ny1-1),
    gt4py_backend=backend,
)
area_mto = Quantity(
    data = metric_terms_o.area.data,
    dims = ("x", "y"),
    units = "degree",
    origin = (nh, nh),
    extent = (nx1-1, ny1-1),
    gt4py_backend=backend,
)

diff_lon = lon_mto.view[:] - lon_edges_rad
diff_lat = lat_mto.view[:] - lat_edges_rad

print()
print("diff lon:", diff_lon.min(), diff_lon.max())
print("diff lat:", diff_lat.min(), diff_lat.max())

diff_lona = lona_mto.view[:] - lon_centers_rad
diff_lata = lata_mto.view[:] - lat_centers_rad

print()
print("diff lon-agrid:", diff_lon.min(), diff_lon.max())
print("diff lat-agrid:", diff_lat.min(), diff_lat.max())

diff_area = area_mto.view[:] - area_fortran.view[:]

print()
print("diff area:", diff_area.min(), diff_area.max())


In [ ]:
if rank == 0:
    fig = plt.figure(figsize=(12, 6))
    fig.patch.set_facecolor("white")
    ax = fig.add_subplot(111, projection=ccrs.Robinson(central_longitude=180))
    ax.set_facecolor(".4")

    # plot computational mesh
    f1 = pcolormesh_cube(
        lat_mt_global.data,
        lon_mt_global.data,
        (area_mt_global.data - area_fortran_global.data),
        cmap="bwr",
        vmin=-1e5,
        vmax=1e5,
    )

    plt.colorbar(f1, label="m2", format="%.1e")

    ax.set_title("Grid area - metric_terms minus fortran")

    plt.show()

In [ ]:
if rank == 0:
    diff = lata_mt_global.data - lata_fortran_global.data
    print(diff.min(), diff.max())

    fig = plt.figure(figsize=(12, 6))
    fig.patch.set_facecolor("white")
    ax = fig.add_subplot(111, projection=ccrs.Robinson(central_longitude=180))
    ax.set_facecolor(".4")

    # plot computational mesh
    f1 = pcolormesh_cube(
        lat_mt_global.data,
        lon_mt_global.data,
        (lata_mt_global.data - lata_fortran_global.data),
        cmap="bwr",
        vmin=-1e-4,
        vmax=1e-4,
    )

    plt.colorbar(f1, label="m2", format="%.1e")

    ax.set_title("Lon on A-grid - metric_terms minus fortran")

    plt.show()

In [ ]:
diff_lon_input = lon_mt.view[:] - lon_edges
diff_lat_input = lat_mt.view[:] - lat_edges
print()
print("diff in lon:", diff_lon_input.min(), diff_lon_input.max())
print("diff in lat:", diff_lat_input.min(), diff_lat_input.max())

In [ ]:
lon_edges.shape

In [ ]:
%%capture
from pace.util.grid import GridData

grid_data = GridData.new_from_metric_terms(metric_terms)

In [ ]:
import numpy as np
from pace.util import Quantity

lon_gd = Quantity(
    data = grid_data.lon.data * 180/np.pi,
    dims = ("x_interface", "y_interface"),
    units = "degree",
    origin = (3, 3),
    extent = (129, 129),
    gt4py_backend=backend,
)

lat_gd = Quantity(
    data = grid_data.lat.data * 180/np.pi,
    dims = ("x_interface", "y_interface"),
    units = "degree",
    origin = (3, 3),
    extent = (129, 129),
    gt4py_backend=backend,
)

area_gd = Quantity(
    data = grid_data.area.data,
    dims = ("x", "y"),
    units = "degree",
    origin = (3, 3),
    extent = (128, 128),
    gt4py_backend=backend,
)

lon_gd_global = communicator.gather(lon_gd)
lat_gd_global = communicator.gather(lat_gd)
area_gd_global = communicator.gather(area_gd)

In [ ]:

if rank == 0:

    fig = plt.figure(figsize=(12, 6))
    fig.patch.set_facecolor("white")
    ax = fig.add_subplot(111, projection=ccrs.Robinson(central_longitude=180))
    ax.set_facecolor(".4")

    # plot computational mesh
    f1 = pcolormesh_cube(
        lat_driver_global.data,
        lon_driver_global.data,
        (area_gd_global.data - area_fortran_global.data),
        cmap="bwr",
        vmin=-1e5,
        vmax=1e5,
    )
    plt.colorbar(f1, format="%.1e", label="m2")

    ax.set_title("Grid data - fortran")

    plt.show()

In [ ]:
# import pace.util.grid.generation
# import importlib
# importlib.reload(pace.util.grid.generation)
# from pace.util.grid.generation import MetricTerms

# metric_terms = MetricTerms(
#         quantity_factory=quantity_factory, communicator=communicator
#     )
# metric_terms = metric_terms.from_lat_lon(lat_edges, lon_edges)



In [ ]:
# import numpy as np
# from pace.util import Quantity

# lon_mt = Quantity(
#     data = metric_terms.lon.data * 180/np.pi,
#     dims = ("x_interface", "y_interface"),
#     units = "degree",
#     origin = (3, 3),
#     extent = (129, 129),
#     gt4py_backend=backend,
# )

# lat_mt = Quantity(
#     data = metric_terms.lat.data * 180/np.pi,
#     dims = ("x_interface", "y_interface"),
#     units = "degree",
#     origin = (3, 3),
#     extent = (129, 129),
#     gt4py_backend=backend,
# )

# lona_mt = Quantity(
#     data = metric_terms.lon_agrid.data * 180/np.pi,
#     dims = ("x", "y"),
#     units = "degree",
#     origin = (3, 3),
#     extent = (128, 128),
#     gt4py_backend=backend,
# )

# lata_mt= Quantity(
#     data = metric_terms.lat_agrid.data * 180/np.pi,
#     dims = ("x", "y"),
#     units = "degree",
#     origin = (3, 3),
#     extent = (128, 128),
#     gt4py_backend=backend,
# )

# area_mt = Quantity(
#     data = metric_terms.area.data,
#     dims = ("x", "y"),
#     units = "degree",
#     origin = (3, 3),
#     extent = (128, 128),
#     gt4py_backend=backend,
# )

# lon_mt_global = communicator.gather(lon_mt)
# lat_mt_global = communicator.gather(lat_mt)
# lona_mt_global = communicator.gather(lona_mt)
# lata_mt_global = communicator.gather(lata_mt)
# area_mt_global = communicator.gather(area_mt)

In [ ]:
# import matplotlib.pyplot as plt
# from cartopy import crs as ccrs
# from fv3viz import pcolormesh_cube

# if rank == 0:
#     fig = plt.figure(figsize=(12, 6))
#     fig.patch.set_facecolor("white")
#     ax = fig.add_subplot(111, projection=ccrs.Robinson())
#     ax.set_facecolor(".4")

#     # plot computational mesh
#     f1 = pcolormesh_cube(
#         lat_driver_global.data,
#         lon_driver_global.data,
#         area_driver_global.data * 2e-11,
#         cmap="viridis",
#         vmin=0,
#         vmax=1,
#     )
#     plt.colorbar(f1, )

#     ax.set_title("Grid area - driver")


#     fig = plt.figure(figsize=(12, 6))
#     fig.patch.set_facecolor("white")
#     ax = fig.add_subplot(111, projection=ccrs.Robinson())
#     ax.set_facecolor(".4")

#     # plot computational mesh
#     f1 = pcolormesh_cube(
#         lat_fortran_global.data,
#         lon_fortran_global.data,
#         area_fortran_global.data * 2e-11,
#         cmap="viridis",
#         vmin=0,
#         vmax=1,
#     )

#     plt.colorbar(f1, )

#     ax.set_title("Grid area - fortran")

#     field = np.zeros((lon_mt_global.data.shape[0], lon_mt_global.data.shape[1]-1, lon_mt_global.data.shape[2]-1))

#     fig = plt.figure(figsize=(12, 6))
#     fig.patch.set_facecolor("white")
#     ax = fig.add_subplot(111, projection=ccrs.Robinson())
#     ax.set_facecolor(".4")

#     # plot computational mesh
#     f1 = pcolormesh_cube(
#         lat_mt_global.data,
#         lon_mt_global.data,
#         area_mt_global.data * 2e-11,
#         cmap="viridis",
#         vmin=0,
#         vmax=1,
#     )

#     plt.colorbar(f1, )

#     ax.set_title("Grid area - metric_terms")

#     plt.show()

In [ ]:
import importlib
import pace.driver.initialization
importlib.reload(pace.driver.initialization)
from pace.driver.initialization import RestartConfig

restart_config = RestartConfig(path=restart_path, start_time=restart_time, fortran_data=fortran_data, fortran_grid=True)
fortran_state = restart_config.get_driver_state(quantity_factory, communicator)

In [ ]:
import numpy as np
from pace.util import Quantity

lon_driver_mt = Quantity(
    data = fortran_state.grid_data.lon.data,
    dims = ("x_interface", "y_interface"),
    units = "degree",
    origin = (3, 3),
    extent = (129, 129),
    gt4py_backend=backend,
)

lat_driver_mt = Quantity(
    data = fortran_state.grid_data.lat.data,
    dims = ("x_interface", "y_interface"),
    units = "degree",
    origin = (3, 3),
    extent = (129, 129),
    gt4py_backend=backend,
)

lona_driver_mt = Quantity(
    data = fortran_state.grid_data.lon_agrid.data,
    dims = ("x", "y"),
    units = "degree",
    origin = (3, 3),
    extent = (128, 128),
    gt4py_backend=backend,
)

lata_driver_mt = Quantity(
    data = fortran_state.grid_data.lat_agrid.data,
    dims = ("x", "y"),
    units = "degree",
    origin = (3, 3),
    extent = (128, 128),
    gt4py_backend=backend,
)

area_driver_mt = Quantity(
    data = fortran_state.grid_data.area.data,
    dims = ("x", "y"),
    units = "degree",
    origin = (3, 3),
    extent = (128, 128),
    gt4py_backend=backend,
)

lon_driver_mt_global = communicator.gather(lon_driver_mt)
lat_driver_mt_global = communicator.gather(lat_driver_mt)
lona_driver_mt_global = communicator.gather(lona_driver_mt)
lata_driver_mt_global = communicator.gather(lata_driver_mt)
area_driver_mt_global = communicator.gather(area_driver_mt)

In [ ]:
import matplotlib.pyplot as plt
from cartopy import crs as ccrs
from fv3viz import pcolormesh_cube

if rank == 0:
    # fig = plt.figure(figsize=(12, 6))
    # fig.patch.set_facecolor("white")
    # ax = fig.add_subplot(111, projection=ccrs.Robinson())
    # ax.set_facecolor(".4")

    # # plot computational mesh
    # f1 = pcolormesh_cube(
    #     lat_driver_global.data,
    #     lon_driver_global.data,
    #     area_driver_global.data * 2e-11,
    #     cmap="viridis",
    #     vmin=0,
    #     vmax=1,
    # )
    # plt.colorbar(f1, )

    # ax.set_title("Grid area - driver")


    # fig = plt.figure(figsize=(12, 6))
    # fig.patch.set_facecolor("white")
    # ax = fig.add_subplot(111, projection=ccrs.Robinson())
    # ax.set_facecolor(".4")

    # # plot computational mesh
    # f1 = pcolormesh_cube(
    #     lat_fortran_global.data,
    #     lon_fortran_global.data,
    #     area_fortran_global.data * 2e-11,
    #     cmap="viridis",
    #     vmin=0,
    #     vmax=1,
    # )

    # plt.colorbar(f1, )

    # ax.set_title("Grid area - fortran")


    fig = plt.figure(figsize=(12, 6))
    fig.patch.set_facecolor("white")
    ax = fig.add_subplot(111, projection=ccrs.Robinson())
    ax.set_facecolor(".4")

    # plot computational mesh
    f1 = pcolormesh_cube(
        lat_driver_mt_global.data,
        lon_driver_mt_global.data,
        area_driver_mt_global.data * 2e-11,
        cmap="viridis",
        vmin=0,
        vmax=1,
    )

    plt.colorbar(f1, )

    ax.set_title("Grid area - driver w/ mod metric_terms")

    plt.show()

In [ ]:
    #print(fortran_state.grid_data.lon.min(), fortran_state.grid_data.lon.max())
print(area_driver_mt.data.min(), area_driver_mt.data.max())

In [ ]:
dir(fortran_state.grid_data)

In [ ]:
from pace.util import QuantityFactory, SubtileGridSizer

backend = "numpy"

sizer = SubtileGridSizer.from_tile_params(
    nx_tile=12,
    ny_tile=12,
    nz=79,
    n_halo=3,
    extra_dim_lengths={},
    layout=layout,
    tile_partitioner=partitioner.tile,
    tile_rank=communicator.tile.rank,
)

quantity_factory = QuantityFactory.from_backend(sizer=sizer, backend=backend)

In [ ]:
from datetime import datetime

restart_path_1 = "/home/ajdas/pace/restart_data/v1.0"
restart_time_1 = datetime.strptime("2016-08-11 00:00:00", "%Y-%m-%d %H:%M:%S")
fortran_data_1 = True
restart_path_4 = "/home/ajdas/pace/restart_data/v4.0"
restart_time_4 = datetime.strptime("2016-08-01 00:15:00", "%Y-%m-%d %H:%M:%S")
fortran_data_4 = True


In [ ]:
%%capture
from pace.driver.initialization import RestartConfig

restart_config_1 = RestartConfig(path=restart_path_1, start_time=restart_time_1, fortran_data=fortran_data_1)
fortran_state_1 = restart_config.get_driver_state(quantity_factory, communicator)
restart_config_4 = RestartConfig(path=restart_path_4, start_time=restart_time_4, fortran_data=fortran_data_4)
fortran_state_4 = restart_config.get_driver_state(quantity_factory, communicator)

In [ ]:
datadir_1 = "/home/ajdas/pace/restart_data/v1.0/"
datadir_4 = "/home/ajdas/pace/restart_data/v4.0/"

In [ ]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
ak_driver_1 = fortran_state_1.grid_data.ak.data
bk_driver_1 = fortran_state_1.grid_data.bk.data
ak_driver_4 = fortran_state_1.grid_data.ak.data
bk_driver_4 = fortran_state_1.grid_data.bk.data

In [ ]:
data = Dataset(datadir_1 + "fv_core.res.nc", "r")
ak_fort_1 = np.array(data["ak"][0])
bk_fort_1 = np.array(data["bk"][0])
data.close()
data = Dataset(datadir_4 + "fv_core.res.nc", "r")
ak_fort_4 = np.array(data["ak"][0])
bk_fort_4 = np.array(data["bk"][0])
data.close()

In [ ]:
fIn_ser_1 = "/home/ajdas/pace/test_data/8.1.1/v1.0/Driver-In.nc"
fIn_ser_4 = "/home/ajdas/pace/test_data/8.1.1/v4.0/Driver-In.nc"
data = Dataset(fIn_ser_1, "r")
ak_ser_1 = np.array(data["ak"][0, 0])
bk_ser_1 = np.array(data["bk"][0, 0])
data.close()
data = Dataset(fIn_ser_4, "r")
ak_ser_4 = np.array(data["ak"][0, 0])
bk_ser_4 = np.array(data["bk"][0, 0])
data.close()

In [ ]:
if rank == 0:
    fig = plt.figure(figsize = (8, 4))
    ax_ak = fig.add_subplot(121)
    ax_bk = fig.add_subplot(122)

    ax_ak.plot(ak_driver_1, np.arange(80), c='k', lw=2, label="driver")
    ax_ak.plot(ak_fort_1, np.arange(80), c='r', lw=2, label="fortran")
    ax_ak.plot(ak_ser_1, np.arange(80), c='b', lw=2, label="serialized")
    ax_bk.plot(bk_driver_1, np.arange(80), c='k', lw=2)
    ax_bk.plot(bk_fort_1, np.arange(80), c='r', lw=2)
    ax_bk.plot(bk_ser_1, np.arange(80), c='b', lw=2)

    ax_ak.set_title("ak - baroclinic 10 day")
    ax_bk.set_title("bk - baroclinic 10 day")

    ax_ak.legend(loc=1)


    fig = plt.figure(figsize = (8, 4))
    ax_ak = fig.add_subplot(121)
    ax_bk = fig.add_subplot(122)

    ax_ak.plot(ak_driver_4, np.arange(80), c='k', lw=2, label="driver")
    ax_ak.plot(ak_fort_4, np.arange(80), c='r', lw=2, label="fortran")
    ax_ak.plot(ak_ser_4, np.arange(80), c='b', lw=2, label="serialized")
    ax_bk.plot(bk_driver_4, np.arange(80), c='k', lw=2)
    ax_bk.plot(bk_fort_4, np.arange(80), c='r', lw=2)
    ax_bk.plot(bk_ser_4, np.arange(80), c='b', lw=2)

    ax_ak.set_title("ak - standard 15 min")
    ax_bk.set_title("bk - standard 15 min")

    ax_ak.legend(loc=1)

    plt.show()

In [ ]:
# lon_tmp = fortran_state.grid_data.lon
# lat_tmp = fortran_state.grid_data.lat

In [ ]:
# import numpy as np
# from pace.util import Quantity

# lon_driver_mt = Quantity(
#     data = fortran_state.grid_data.lon.data * 180/np.pi,
#     dims = ("x_interface", "y_interface"),
#     units = "degree",
#     origin = (3, 3),
#     extent = (129, 129),
#     gt4py_backend=backend,
# )

# lat_driver_mt = Quantity(
#     data = fortran_state.grid_data.lat.data * 180/np.pi,
#     dims = ("x_interface", "y_interface"),
#     units = "degree",
#     origin = (3, 3),
#     extent = (129, 129),
#     gt4py_backend=backend,
# )

# lona_driver_mt = Quantity(
#     data = fortran_state.grid_data.lon_agrid.data * 180/np.pi,
#     dims = ("x", "y"),
#     units = "degree",
#     origin = (3, 3),
#     extent = (128, 128),
#     gt4py_backend=backend,
# )

# lata_driver_mt = Quantity(
#     data = fortran_state.grid_data.lat_agrid.data * 180/np.pi,
#     dims = ("x", "y"),
#     units = "degree",
#     origin = (3, 3),
#     extent = (128, 128),
#     gt4py_backend=backend,
# )

# area_driver_mt = Quantity(
#     data = fortran_state.grid_data.area.data,
#     dims = ("x", "y"),
#     units = "degree",
#     origin = (3, 3),
#     extent = (128, 128),
#     gt4py_backend=backend,
# )

# lon_driver_mt_global = communicator.gather(lon_driver_mt)
# lat_driver_mt_global = communicator.gather(lat_driver_mt)
# lona_driver_mt_global = communicator.gather(lona_driver_mt)
# lata_driver_mt_global = communicator.gather(lata_driver_mt)
# area_driver_mt_global = communicator.gather(area_driver_mt)

In [ ]:
# import matplotlib.pyplot as plt
# from cartopy import crs as ccrs
# from fv3viz import pcolormesh_cube

# if rank == 0:
#     fig = plt.figure(figsize=(12, 6))
#     fig.patch.set_facecolor("white")
#     ax = fig.add_subplot(111, projection=ccrs.Robinson())
#     ax.set_facecolor(".4")

#     # plot computational mesh
#     f1 = pcolormesh_cube(
#         lat_driver_global.data,
#         lon_driver_global.data,
#         area_driver_global.data * 2e-11,
#         cmap="viridis",
#         vmin=0,
#         vmax=1,
#     )
#     plt.colorbar(f1, )

#     ax.set_title("Grid area - driver")


#     fig = plt.figure(figsize=(12, 6))
#     fig.patch.set_facecolor("white")
#     ax = fig.add_subplot(111, projection=ccrs.Robinson())
#     ax.set_facecolor(".4")

#     # plot computational mesh
#     f1 = pcolormesh_cube(
#         lat_fortran_global.data,
#         lon_fortran_global.data,
#         area_fortran_global.data * 2e-11,
#         cmap="viridis",
#         vmin=0,
#         vmax=1,
#     )

#     plt.colorbar(f1, )

#     ax.set_title("Grid area - fortran")


#     fig = plt.figure(figsize=(12, 6))
#     fig.patch.set_facecolor("white")
#     ax = fig.add_subplot(111, projection=ccrs.Robinson())
#     ax.set_facecolor(".4")

#     # plot computational mesh
#     f1 = pcolormesh_cube(
#         lat_mt_global.data,
#         lon_mt_global.data,
#         area_mt_global.data * 2e-11,
#         cmap="viridis",
#         vmin=0,
#         vmax=1,
#     )

#     plt.colorbar(f1, )

#     ax.set_title("Grid area - metric_terms")

#     fig = plt.figure(figsize=(12, 6))
#     fig.patch.set_facecolor("white")
#     ax = fig.add_subplot(111, projection=ccrs.Robinson())
#     ax.set_facecolor(".4")

#     # plot computational mesh
#     f1 = pcolormesh_cube(
#         lat_driver_mt_global.data,
#         lon_driver_mt_global.data,
#         area_driver_mt_global.data * 2e-11,
#         cmap="viridis",
#         vmin=0,
#         vmax=1,
#     )

#     plt.colorbar(f1, )

#     ax.set_title("Grid area - driver w/ mod metric_terms")

#     plt.show()